In [0]:
import pandas as pd
import pickle
from datetime import datetime

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys

dir_visualization = 'drive/My Drive/Dashboard visualization/library'

sys.path.append(dir_visualization)

#import function file
import load_data as f

In [0]:
path = 'drive/My Drive/Dashboard visualization/data/'

In [0]:
food_file = 'drive/My Drive/Dashboard visualization/data/foodpandas.csv'
beneficiaris_file = 'drive/My Drive/Dashboard visualization/data/beneficiaris.csv'
poblacio_dir = 'drive/My Drive/Dashboard visualization/data/'

In [0]:
food_df = pd.read_csv(food_file,sep = ',')
beneficiaris_df = pd.read_csv(beneficiaris_file,sep = ',')

In [0]:
beneficiaris_df.head()
food_df.head()

,Unnamed: 0,IDComarca,IDMunicipi,Quantitat,Any,NomComarca,NomMunicipi,IDMacrofamilia,NomMacrofamilia
0,0,6,80446,453.38,2017-01-01,Anoia,Capellades,H,Fruites i verdures fresques
1,1,6,80446,1727.06,2016-01-01,Anoia,Capellades,H,Fruites i verdures fresques
2,2,6,80446,683.08,2015-01-01,Anoia,Capellades,H,Fruites i verdures fresques
3,3,6,80446,132.00,2017-01-01,Anoia,Capellades,H,Fruites i verdures fresques
4,4,6,81022,36364.13,2017-01-01,Anoia,Igualada,H,Fruites i verdures fresques


In [0]:
food_comarca_any_df = food_df[['NomComarca','Any','Quantitat']].groupby(['NomComarca','Any']).sum()

In [0]:
beneficiaris_df.loc[beneficiaris_df.NomComarca == 'maresme','NomComarca'] = 'Maresme'

In [0]:
beneficiaris_quantitat_df = food_comarca_any_df.merge(beneficiaris_df, left_index=True, right_on=['NomComarca','Any'], how='right')

In [0]:
beneficiaris_quantitat_df.loc[beneficiaris_quantitat_df.Quantitat.isna(),'Quantitat'] = 0

In [0]:
beneficiaris_quantitat_df.Any = beneficiaris_quantitat_df.Any.apply(lambda x: int(x[:4]))

In [0]:
beneficiaris_quantitat_df.head()

,Quantitat,Beneficiaris,NomComarca,Any
0,13118.34,209636,Anoia,2008
1,30792.98,257788,Anoia,2009
2,58736.13,321905,Anoia,2010
3,62936.32,416867,Anoia,2011
4,84869.92,378994,Anoia,2012


In [0]:
#beneficiaris_quantitat_df[['NomComarca','Beneficiaris','Any']].groupby(['NomComarca','Any']).max()
beneficiaris_quantitat_df.head()

,Quantitat,Beneficiaris,NomComarca,Any
0,13118.34,209636,Anoia,2008
1,30792.98,257788,Anoia,2009
2,58736.13,321905,Anoia,2010
3,62936.32,416867,Anoia,2011
4,84869.92,378994,Anoia,2012


In [0]:
poblacio_df = l.load_poblacio(poblacio_dir,stack=True)
poblacio_df.head()

,NomComarca,Any,Poblacio
0,Alt Camp,2004,38132
1,Alt Camp,2005,39247
2,Alt Camp,2006,40677
3,Alt Camp,2007,42227
4,Alt Camp,2008,43805


In [0]:
beneficiaris_quantitat_df = beneficiaris_quantitat_df.merge(poblacio_df, how='left', left_on = ['NomComarca','Any'], right_on = ['NomComarca','Any'])

In [0]:
beneficiaris_quantitat_df.NomComarca == 'Moianès'

In [0]:
beneficiaris_quantitat_df['Quantitat_Normalitzada'] = beneficiaris_quantitat_df.Quantitat * 1000 / beneficiaris_quantitat_df.Poblacio
beneficiaris_quantitat_df['Beneficiaris_Normalitzat'] = beneficiaris_quantitat_df.Beneficiaris * 1000 / beneficiaris_quantitat_df.Poblacio
beneficiaris_quantitat_df['Quantitat_Normalitzada'] = beneficiaris_quantitat_df['Quantitat_Normalitzada'].apply(lambda x: round(x,2))
beneficiaris_quantitat_df.loc[beneficiaris_quantitat_df.Beneficiaris_Normalitzat.isna(),['Beneficiaris_Normalitzat','Quantitat_Normalitzada','Poblacio']]= 0
beneficiaris_quantitat_df['Beneficiaris_Normalitzat'] = beneficiaris_quantitat_df[~beneficiaris_quantitat_df.Beneficiaris_Normalitzat.isna()].Beneficiaris_Normalitzat.apply(lambda x: round(x,0))
beneficiaris_quantitat_df.head()

,Quantitat,Beneficiaris,NomComarca,Any,Poblacio,Quantitat_Normalitzada,Beneficiaris_Normalitzat
0,13118.34,209636,Anoia,2008,114090.0,114.98,1837.0
1,30792.98,257788,Anoia,2009,116620.0,264.05,2210.0
2,58736.13,321905,Anoia,2010,117777.0,498.71,2733.0
3,62936.32,416867,Anoia,2011,118298.0,532.02,3524.0
4,84869.92,378994,Anoia,2012,118495.0,716.23,3198.0


Inconsistències Dataset:

- En el dataset de beneficiarisComarcas.pkl les dades venen entre 2008 i 2017, mentres que en el dataset de food_pandas.csv les dades venen entre 2007 i 2017. -> Prescindim del any 2007
- En el dataset de food_pandas.csv per a la Comarca del Bages falta el any 2008 -> Quantitat = 0
- En el dataset de beneficiarisComarcas.pkl el camp NomComarca presenta diversos valors per a la mateixa comarca (Maresme,maresme)

## Balls Beneficiaris_Quantitat + Year_Slider

In [0]:
import altair as alt
from altair import datum

In [0]:
import sys

dir_visualization = 'drive/My Drive/Dashboard visualization/library'
path = 'drive/My Drive/Dashboard visualization/data/'

sys.path.append(dir_visualization)

#import function file
import load_data as f

beneficiaris_quantitat_df = f.load_beneficiaris_comarca(path)

In [18]:

slider = alt.binding_range(min=2008, max=2017, step=1)

select_year = alt.selection_single(name='Filtre', fields=['Any'], bind=slider, empty='all')

balls_bqy = alt.Chart(beneficiaris_quantitat_df).mark_circle(
    opacity=0.8,
    stroke='black',
    strokeWidth=1
).encode(
    x = alt.X('Quantitat_Normalitzada:Q', scale=alt.Scale(domain=(0, 3000)), title = 'Quantitat cada 1000 habitants'),
    y = alt.Y('Beneficiaris_Normalitzat:Q', scale=alt.Scale(domain=(0, 22000)), title = 'Beneficiaris cada 1000 habitants'),
    color = alt.Color('NomComarca:N',title='Comarca',legend=None),
    size = alt.Size('Poblacio:Q', scale=alt.Scale(range=[100, 10000]), legend=None),
    tooltip= alt.Tooltip('NomComarca')
).properties(
    height=450,
    width=750
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).add_selection(
    select_year
).transform_filter(
    select_year
)

balls_bqy

Chart({
  config: Config({
    axis: AxisConfig({
      grid: False
    }),
    view: ViewConfig({
      strokeWidth: 0
    })
  }),
  data:        Quantitat  Beneficiaris         NomComarca   Any  Poblacio  \
  0      13118.340        209636              Anoia  2008  114090.0   
  1      30792.980        257788              Anoia  2009  116620.0   
  2      58736.130        321905              Anoia  2010  117777.0   
  3      62936.320        416867              Anoia  2011  118298.0   
  4      84869.920        378994              Anoia  2012  118495.0   
  5     133599.970        728082              Anoia  2013  118045.0   
  6     220768.930       1009281              Anoia  2014  117465.0   
  7     256576.980        926150              Anoia  2015  117117.0   
  8     243465.470        926150              Anoia  2016  117361.0   
  9     257133.087        794240              Anoia  2017  118405.0   
  10         0.000          3630              Bages  2008  180361.0   
  11      1046.100          5670              Bages  2009  183576.0   
  12     51122.870        399101              Bages  2010  184498.0   
  13     76346.020        913869              Bages  2011  185289.0   
  14    121231.180        825551              Bages  2012  185416.0   
  15    137989.100       1104987              Bages  2013  184182.0   
  16    129712.860       1168681              Bages  2014  182795.0   
  17    188872.240       1358217              Bages  2015  173193.0   
  18    157559.030       1358217              Bages  2016  173143.0   
  19    159385.817        560482              Bages  2017  173724.0   
  20    320329.170       6081941     Baix Llobregat  2008  773279.0   
  21    350984.660       4655335     Baix Llobregat  2009  786378.0   
  22    408242.170       4589509     Baix Llobregat  2010  793204.0   
  23    470885.040       5781788     Baix Llobregat  2011  799471.0   
  24    560885.290       5882402     Baix Llobregat  2012  801659.0   
  25    952094.860      11796935     Baix Llobregat  2013  800193.0   
  26   1169607.000      12090045     Baix Llobregat  2014  797677.0   
  27   1486446.580      11760442     Baix Llobregat  2015  798879.0   
  28   1502888.430      11760442     Baix Llobregat  2016  802520.0   
  29   1113217.758      10230800     Baix Llobregat  2017  807796.0   
  ..           ...           ...                ...   ...       ...   
  80     15091.280         39902              Osona  2008  149410.0   
  81     54226.690         42150              Osona  2009  151710.0   
  82    175912.370       1134450              Osona  2010  152917.0   
  83    172086.650        886303              Osona  2011  153779.0   
  84    169017.960       1068792              Osona  2012  154042.0   
  85    297910.310       1173083              Osona  2013  153563.0   
  86    337678.150        902092              Osona  2014  152560.0   
  87    374238.090        861696              Osona  2015  152582.0   
  88    257458.170        861696              Osona  2016  153471.0   
  89    278869.260       1222786              Osona  2017  154783.0   
  90    438301.980       6131721  Vallès Occidental  2008  856345.0   
  91    759532.580       6824019  Vallès Occidental  2009  872754.0   
  92    959411.640       7767519  Vallès Occidental  2010  882324.0   
  93    988596.010       9113821  Vallès Occidental  2011  889924.0   
  94   1189810.840      10507981  Vallès Occidental  2012  894813.0   
  95   1782111.830      12066319  Vallès Occidental  2013  894638.0   
  96   2048063.420      11243856  Vallès Occidental  2014  893038.0   
  97   2342693.990      10836815  Vallès Occidental  2015  895166.0   
  98   2307322.490      10836815  Vallès Occidental  2016  900516.0   
  99   2154737.664       9066838  Vallès Occidental  2017  908026.0   
  100    54762.120       2088647    Vallès Oriental  2008  383146.0   
  101   105604.620       1337564    Vallès Oriental  2009  390880.0   
  102   143710.700       15

In [0]:
balls_bqy.save('drive/My Drive/Dashboard visualization/dashboard/balls_bqy.json')